In [1]:
from sklearn.decomposition import IncrementalPCA
import os
import sklearn
from skimage import io
import numpy as np 
import matplotlib.pyplot as plt
import cv2
import sys
sys.path.append('../')
import torch as tc
from src.dataset.dataloader import *


In [2]:
import importlib
importlib.reload(sys.modules['src.dataset.dataloader'])
from src.dataset.dataloader import *

In [3]:
img_dir = "../data/raw"
batch_size = 10000
size = 425000
dataset = CREDO_Dataset(img_dir, True)
dataloader = tc.utils.data.DataLoader(dataset, batch_size = batch_size, num_workers=2)

In [4]:
import time
import pickle as pkl
dimensions = 60

#https://scikit-cuda.readthedocs.io/en/latest/generated/skcuda.linalg.PCA.html

sklearn_pca = IncrementalPCA(n_components=dimensions, batch_size=10000)
#sklearn_pca = pkl.load(open("../models/model_pca_big.pkl",'rb'))
tim = time.time()
for index, data in enumerate(dataloader):
    print(time.time() - tim)
    if index > -1:
        time1 = time.time()

        x, y, z = data[0].shape
        data_chunk = data[0].reshape(x, y*z)

        time2 = time.time()

        print(f"Czas reshape: {time2-time1}")
        sklearn_pca.partial_fit(data_chunk)

        time3 = time.time()
        print(f"Czas PCA: {time3-time2}")

        pkl.dump(sklearn_pca, open("../models/model_pca_big.pkl","wb"))

        print(f"DONE: {(index+1)*x}/{size}")
        print(sum(sklearn_pca.explained_variance_ratio_))

        #Estimated waiting time: ~40 minutes


131.14944338798523
Czas reshape: 0.0
Czas PCA: 35.431196212768555
DONE: 10000/425000
166.59027981758118
Czas reshape: 0.0181577205657959
Czas PCA: 36.28709006309509
DONE: 20000/425000
220.09056162834167
Czas reshape: 0.017001867294311523
Czas PCA: 31.086997747421265
DONE: 30000/425000
251.20356011390686
Czas reshape: 0.017000436782836914
Czas PCA: 32.898754358291626
DONE: 40000/425000
296.5943202972412
Czas reshape: 0.016999483108520508
Czas PCA: 32.97430491447449
DONE: 50000/425000
329.59730291366577
Czas reshape: 0.019220590591430664
Czas PCA: 35.633469104766846
DONE: 60000/425000
380.94127798080444
Czas reshape: 0.016998767852783203
Czas PCA: 32.79642677307129
DONE: 70000/425000
413.7627091407776
Czas reshape: 0.024003028869628906
Czas PCA: 34.31166338920593
DONE: 80000/425000
468.39160680770874
Czas reshape: 0.020998716354370117
Czas PCA: 34.227784395217896
DONE: 90000/425000
502.64938974380493
Czas reshape: 0.01701068878173828
Czas PCA: 34.62071919441223
DONE: 100000/425000
560.91

In [5]:
print(sum(sklearn_pca.explained_variance_ratio_))

#8 cech wyjaśnia już aż 96,08% ! (nie pamietam dokładnej metody preporecssingu)
#30 cech to dopiero 87% dla progowania (dla dużego zbioru)
#60 cech to dopiero 90,2% dla całego nowego prepro (bez removedust) (dla dużego zbioru)
#60 cech to dopiero 86,16% dla całego starego prepro (dla dużego zbioru)
#60 cech to dopiero 90,7% dla całego nowego prepro (z removedust) (dla dużego zbioru)

0.9073288896358433
